# Modelo no supervisado de clasificación de texto

## Introducción

En este notebook se presenta un modelo no supervisado de clasificación de texto. El modelo se basa en el uso de embeddings de palabras y clustering. Se utiliza el algoritmo de clustering KMeans para agrupar los textos en clusters. 



In [ ]:
### Librerias necesarias Doc2Vec

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Librerias necesarias para el preprocesamiento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')


### Otras librerias necesarias
import numpy as np
import pandas as pd
import re
import os

## 2. Cargar el texto

El dataset "es_tweets_laboral" de la colección "somosnlp-hackathon-2022" en Hugging Face está diseñado específicamente para el análisis de texto relacionado con temas laborales en español. Este dataset contiene tuits que abordan temas laborales, y es ideal para tareas de clasificación de texto, análisis de sentimientos, y otras aplicaciones de procesamiento de lenguaje natural (NLP) enfocadas en el ámbito laboral.

**Características del Dataset:**
- *Contenido*: Incluye tuits en español relacionados con temas laborales, como empleo, condiciones de trabajo, y derechos laborales.
- *Etiquetas*: Los tuits pueden estar etiquetados según el tema o el sentimiento, lo que facilita su uso en tareas de clasificación supervisada.
- *Aplicaciones*: Este dataset es útil para construir modelos que analicen la percepción de los usuarios sobre temas laborales, detectar tendencias en el mercado laboral, o identificar problemas comunes en el ámbito laboral.

El dataset es parte de un esfuerzo colaborativo durante el Hackathon de SomosNLP en 2022, que busca fomentar el desarrollo de tecnologías de procesamiento de lenguaje natural en español.



In [ ]:
### Carga dataset desde huggingface

from datasets import load_dataset
import tqdm as notebook_tqdm

# Cargando el dataset "es_tweets_laboral" desde Hugging Face
dataset = load_dataset("somosnlp-hackathon-2022/es_tweets_laboral")

# Explorando el contenido del dataset
print(dataset)


In [ ]:
train = dataset['train']
test = dataset['test']

# Explorando el contenido de los datos de entrenamiento

train_df = train.to_pandas()
test_df = test.to_pandas()
print("Shape of train data: ", train_df.shape)
print("Shape of test data: ", test_df.shape)

train_df.head()

In [ ]:
test_df.head()

## 3. Preprocesamiento

Empezamos por cargar el dataset y realizar un preprocesamiento básico de los textos. En este caso, se eliminan las menciones a usuarios, los enlaces, y los caracteres especiales. Además, se convierten los textos a minúsculas y se eliminan las stopwords.

In [ ]:
########## Preprocesamiento de texto ###############

def preprocess_text(text):
    # Eliminando caracteres especiales y números
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚ\s]', '', text, re.I|re.A)
    # Convertir a minúsculas
    text = text.lower()
    # eliminando stopwords
    stop_words = set(stopwords.words('spanish'))
    word_tokens = word_tokenize(text)
    text = [i for i in word_tokens if not i in stop_words]
    text = ' '.join(text)
    return text

# Aplicando la función de preprocesamiento a los datos de entrenamiento y prueba

train_df['text_pre'] = train_df['text'].apply(preprocess_text)

test_df['text_pre'] = test_df['text'].apply(preprocess_text)

train_df.head()

## 4. Modelo de Clasificación de Texto

Una vez que hemos preprocesado los textos, podemos aplicar un modelo de clasificación no supervisado para agruparlos en categorías o clusters. En este caso, utilizaremos el algoritmo de clustering KMeans para agrupar los textos en clusters. AUnque primero debemos convertir los textos en vectores numéricos utilizando embeddings de palabras.


In [ ]:
################## Entrenamiento de Word2Vec ####################

# Tokenizando el texto

train_df['text_tokens'] = train_df['text_pre'].apply(lambda x: x.split())
test_df['text_tokens'] = test_df['text_pre'].apply(lambda x: x.split())

train_df.head()

In [ ]:
# Entrenando el modelo Doc2Vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_df['text_tokens'])]

# Definiendo el modelo Doc2Vec

model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

# Guardando el modelo entrenado

model.save("doc2vec.model")


In [ ]:
# Cargando el modelo entrenado

model = Doc2Vec.load("doc2vec.model")

# Obteniendo el vector de una palabra

model.wv['trabajo']

# Obteniendo las palabras más similares a una palabra

model.wv.most_similar('trabajo')


In [ ]:
# Obteniendo la similitud entre dos palabras

model.wv.similarity('trabajo', 'empleo')


In [ ]:
#### Hacemos un clustering de los tweets con KMeans

from sklearn.cluster import KMeans

# Obteniendo los vectores de los tweets

vectors = [model.infer_vector(doc) for doc in train_df['text_tokens']]

# Definiendo el modelo KMeans

kmeans = KMeans(n_clusters=5, random_state=0)

# Entrenando el modelo KMeans

kmeans.fit(vectors)

# Obteniendo las etiquetas de los clusters

train_df['cluster'] = kmeans.labels_

# Explorando los clusters

train_df['cluster'].value_counts()



In [ ]:
### Veamos los tweets de un cluster

pd.set_option('display.max_colwidth', None)

train_df[train_df['cluster'] == 0]['text'].head(10)

In [ ]:
### Veamos el centroide de cada cluster

for i in range(5):
    print("Cluster ", i)
    print(model.wv.most_similar(positive=[kmeans.cluster_centers_[i]], topn=10))
    print("\n\n")